In [1]:
# Configure Drive or Jupyter notebook -- only runs when first loaded
if "CONFIG_DONE" not in globals():
    # Need to mount drive and clone repo to access data and functions
    try:
        from google.colab import drive  # type: ignore

        IN_COLAB = True

        # clone repo
        !git clone https://github.com/doctorsmylie/mtg-draft-agent
        %cd mtg-draft-agent

    except ModuleNotFoundError:
        IN_COLAB = False

    # Finish configuration -- also configures notebook outside of Colab
    %run "project_path.ipynb"
else:
    print("Config done already")

Starting config...
Running in Colab? No

Configuring Jupyter Notebook...
cd to the repo's base directory...

BASE_PATH =  /home/mrgomez/Documents/Erdos/DL_2025/mtg-draft-agent
DATA_FOLDER = /home/mrgomez/Documents/Erdos/DL_2025/mtg-draft-agent/MTGdraft
BASE_PATH == os.getcwd(): True

Configuration done


In [17]:
import numpy as np
import pandas as pd

# For managing paths
import pathlib

from time import time

from itertools import product

# Start exploring
See the data in 'draft_data...' files.

In [3]:
# Folder containing all the data
print(DATA_FOLDER)

# Expansion code
expansion = "DSK"

/home/mrgomez/Documents/Erdos/DL_2025/mtg-draft-agent/MTGdraft


In [4]:
draftfilename = "draft_data_public." + expansion + ".PremierDraft.csv.gz"
draft_file = pathlib.Path(DATA_FOLDER, expansion, draftfilename)

draftdata = pd.read_csv(draft_file, compression="gzip", nrows=10000)
num_cols = draftdata.shape[1]

In [5]:
for col in draftdata.columns:
    print(col)
print()

print(draftdata.shape)

expansion
event_type
draft_id
draft_time
rank
event_match_wins
event_match_losses
pack_number
pick_number
pick
pick_maindeck_rate
pick_sideboard_in_rate
pack_card_Abandoned Campground
pack_card_Abhorrent Oculus
pack_card_Acrobatic Cheerleader
pack_card_Altanak, the Thrice-Called
pack_card_Anthropede
pack_card_Appendage Amalgam
pack_card_Arabella, Abandoned Doll
pack_card_Attack-in-the-Box
pack_card_Balemurk Leech
pack_card_Balustrade Wurm
pack_card_Baseball Bat
pack_card_Bashful Beastie
pack_card_Bear Trap
pack_card_Beastie Beatdown
pack_card_Bedhead Beastie
pack_card_Betrayer's Bargain
pack_card_Blazemire Verge
pack_card_Bleeding Woods
pack_card_Boilerbilges Ripper
pack_card_Bottomless Pool // Locker Room
pack_card_Break Down the Door
pack_card_Broodspinner
pack_card_Cackling Slasher
pack_card_Cathartic Parting
pack_card_Cautious Survivor
pack_card_Central Elevator // Promising Stairs
pack_card_Chainsaw
pack_card_Charred Foyer // Warped Space
pack_card_Clammy Prowler
pack_card_Clockwo

Great, it looks like each row has the cards that were available to the player, the card that was picked, and the cards that the player already picked. Let's check out specific rows.

In [6]:
row = 2
for idx in range(num_cols):
    print(draftdata.columns[idx], end=": ")
    print(draftdata.iloc[row, idx])

expansion: DSK
event_type: PremierDraft
draft_id: 53401b113a4f425fa26e60edd314dd27
draft_time: 2024-09-24 18:48:56
rank: diamond
event_match_wins: 1
event_match_losses: 3
pack_number: 0
pick_number: 2
pick: Fear of Being Hunted
pick_maindeck_rate: 1.0
pick_sideboard_in_rate: 0.0
pack_card_Abandoned Campground: 0
pack_card_Abhorrent Oculus: 0
pack_card_Acrobatic Cheerleader: 0
pack_card_Altanak, the Thrice-Called: 0
pack_card_Anthropede: 0
pack_card_Appendage Amalgam: 0
pack_card_Arabella, Abandoned Doll: 0
pack_card_Attack-in-the-Box: 0
pack_card_Balemurk Leech: 1
pack_card_Balustrade Wurm: 0
pack_card_Baseball Bat: 0
pack_card_Bashful Beastie: 1
pack_card_Bear Trap: 0
pack_card_Beastie Beatdown: 0
pack_card_Bedhead Beastie: 0
pack_card_Betrayer's Bargain: 0
pack_card_Blazemire Verge: 0
pack_card_Bleeding Woods: 1
pack_card_Boilerbilges Ripper: 0
pack_card_Bottomless Pool // Locker Room: 0
pack_card_Break Down the Door: 0
pack_card_Broodspinner: 0
pack_card_Cackling Slasher: 0
pack_car

As expected, we have the card that was picked in `pick`. The columns `pack_card` are booleans indicating if the card was there, and `pool` are the cards chosen in previous rounds. See the following specific example.

In [7]:
cols = [
    "draft_id",
    "pack_number",
    "pick_number",
    "pick",
    # Available cards (only show a couple)
    "pack_card_Infernal Phantom",
    "pack_card_Floodpits Drowner",
    "pack_card_Fear of Being Hunted",
    # Picked cards (only show a couple)
    "pool_Infernal Phantom",
    "pool_Floodpits Drowner",
    "pool_Fear of Being Hunted",
]
rows = [0, 1, 2]

for row in rows:
    for col in cols:
        print(col, end=": ")
        print(draftdata.at[row, col])
    print()

draft_id: 53401b113a4f425fa26e60edd314dd27
pack_number: 0
pick_number: 0
pick: Infernal Phantom
pack_card_Infernal Phantom: 1
pack_card_Floodpits Drowner: 0
pack_card_Fear of Being Hunted: 0
pool_Infernal Phantom: 0
pool_Floodpits Drowner: 0
pool_Fear of Being Hunted: 0

draft_id: 53401b113a4f425fa26e60edd314dd27
pack_number: 0
pick_number: 1
pick: Floodpits Drowner
pack_card_Infernal Phantom: 0
pack_card_Floodpits Drowner: 1
pack_card_Fear of Being Hunted: 0
pool_Infernal Phantom: 1
pool_Floodpits Drowner: 0
pool_Fear of Being Hunted: 0

draft_id: 53401b113a4f425fa26e60edd314dd27
pack_number: 0
pick_number: 2
pick: Fear of Being Hunted
pack_card_Infernal Phantom: 0
pack_card_Floodpits Drowner: 0
pack_card_Fear of Being Hunted: 1
pool_Infernal Phantom: 1
pool_Floodpits Drowner: 1
pool_Fear of Being Hunted: 0



# Get list of cards

Our model will need:
1. List of available options (a list `options`)
2. Card chosen (a list `chosen`)

Our current dataframe has that data but in a 0-1 encoding. We could instead use card indices and an embedding layer. We also need to filter out incomplete drafts.

Let's look at a couple of draft_ids:

In [8]:
# Get unique draft ids
draft_ids = draftdata["draft_id"].unique()
print("Number of rows:", draftdata.shape[0])
print("Number of unique ids:", draft_ids.shape)

Number of rows: 10000
Number of unique ids: (239,)


In [9]:
# Find which ids are complete
picks_per_id = draftdata.value_counts("draft_id")

with pd.option_context("display.max_rows", None):
    print(picks_per_id)

draft_id
0041b2ed5a16424ebd63fcbede00776f    42
004fc7f51d7f47fe92267908072ae788    42
00c08bf90a7c44d2a1d2daf748cedd54    42
02821de13f12445f87366450bdee588e    42
02ae214587434247a6e3d07ffe428d47    42
034f231f793f4b788fbb51007ee72fef    42
0444b1d527894b7da230a9c5970c2f18    42
056b8c6f98bd4bf08ccd236a3c598973    42
05b16fd8051248e38af5fdd5b79d52ab    42
088b770373e34cd1b676c652bdcce387    42
094bdff65a0c4f538da2603ab10fa157    42
0b2584a9642048ca8557e3622289b335    42
0b752016c35b4589bbce11f70c6d3fd3    42
0d38f26423f54d5ca1b05bfe553162e7    42
0eb278241c0e4a4185230b26225926b1    42
102627f0321b4f4f890c33de6b982e4b    42
12cdd529fa524e1e85d25feaa8ca5f16    42
15a2a2d241d14fca90bc0d02c57d759b    42
1667fb270bea4c23a48be3b9f9e768a2    42
168656924b414600a4598e3ad6af9ee2    42
16ae403277924cfbaf31a0677e9d6c46    42
1a017466735b44bf9970ebd523ac3c16    42
1b100a905d444d86bef2ffb470a022e8    42
1ba47e2c02db437cbc8bb95bc06b6259    42
1c67a887ee4f422685bb6e95971b0b6b    42
1c996c7fdea74418

Since we only loaded about 10,000 rows, we have an incomplete draft. The others, however, seem complete. Let's look at the picks of a single draft_id

In [10]:
id = draft_ids[0]

cols = ["pack_number", "pick_number", "pick"]

draftdata.loc[draftdata["draft_id"] == id, cols]

,pack_number,pick_number,pick
0,0,0,Infernal Phantom
1,0,1,Floodpits Drowner
2,0,2,Fear of Being Hunted
3,0,3,Fear of Being Hunted
4,0,4,Undead Sprinter
5,0,5,Dashing Bloodsucker
6,0,6,Grab the Prize
7,0,7,Glassworks // Shattered Yard
8,0,8,Impossible Inferno
9,0,9,Ripchain Razorkin


We'd like a dictionary that assigns indices to card names. First, we need a list of all cards. We'll get them from the `pack_card` and `pool` column names. Any one of the two should have enough data for our dict, but we'll do it twice for a sanity check.

In [11]:
card_names_packs = draftdata.filter(regex="pack_card").columns
card_names_packs = list(card_names_packs)
# Remove annotation (pack_card_)
card_names_packs = [name.replace("pack_card_", "") for name in card_names_packs]

card_names_pool = draftdata.filter(regex="pool").columns
card_names_pool = list(card_names_pool)
# Remove annotation (pool_)
card_names_pool = [name.replace("pool_", "") for name in card_names_pool]

# Check that the results are the same
print("Card lists match?:", card_names_packs == card_names_pool)

Card lists match?: True


Since the list of cards in the `pack_card` and `pool` columns are the same, we can use either. We'll print all cards for reference.

In [12]:
card_names = draftdata.filter(regex="pack_card").columns
card_names = list(card_names)
# Remove annotation (pack_card_)
card_names = [name.replace("pack_card_", "") for name in card_names]

for name in card_names:
    print(name)

Abandoned Campground
Abhorrent Oculus
Acrobatic Cheerleader
Altanak, the Thrice-Called
Anthropede
Appendage Amalgam
Arabella, Abandoned Doll
Attack-in-the-Box
Balemurk Leech
Balustrade Wurm
Baseball Bat
Bashful Beastie
Bear Trap
Beastie Beatdown
Bedhead Beastie
Betrayer's Bargain
Blazemire Verge
Bleeding Woods
Boilerbilges Ripper
Bottomless Pool // Locker Room
Break Down the Door
Broodspinner
Cackling Slasher
Cathartic Parting
Cautious Survivor
Central Elevator // Promising Stairs
Chainsaw
Charred Foyer // Warped Space
Clammy Prowler
Clockwork Percussionist
Collected Company
Come Back Wrong
Commune with Evil
Conductive Machete
Coordinated Clobbering
Cracked Skull
Creeping Peeper
Cryptid Inspector
Cult Healer
Cursed Recording
Cursed Windbreaker
Cynical Loner
Daggermaw Megalodon
Damnation
Dashing Bloodsucker
Dazzling Theater // Prop Room
Defiant Survivor
Defiled Crypt // Cadaver Lab
Demonic Counsel
Derelict Attic // Widow's Walk
Dissection Tools
Disturbing Mirth
Diversion Specialist
Doll

And now, writing the dict is a piece of cake.

In [13]:
card_to_idx = {}
idx_to_card = {}

for idx, card in enumerate(card_names):
    card_to_idx[card] = idx
    idx_to_card[idx] = card

In [14]:
card_to_idx

{'Abandoned Campground': 0,
 'Abhorrent Oculus': 1,
 'Acrobatic Cheerleader': 2,
 'Altanak, the Thrice-Called': 3,
 'Anthropede': 4,
 'Appendage Amalgam': 5,
 'Arabella, Abandoned Doll': 6,
 'Attack-in-the-Box': 7,
 'Balemurk Leech': 8,
 'Balustrade Wurm': 9,
 'Baseball Bat': 10,
 'Bashful Beastie': 11,
 'Bear Trap': 12,
 'Beastie Beatdown': 13,
 'Bedhead Beastie': 14,
 "Betrayer's Bargain": 15,
 'Blazemire Verge': 16,
 'Bleeding Woods': 17,
 'Boilerbilges Ripper': 18,
 'Bottomless Pool // Locker Room': 19,
 'Break Down the Door': 20,
 'Broodspinner': 21,
 'Cackling Slasher': 22,
 'Cathartic Parting': 23,
 'Cautious Survivor': 24,
 'Central Elevator // Promising Stairs': 25,
 'Chainsaw': 26,
 'Charred Foyer // Warped Space': 27,
 'Clammy Prowler': 28,
 'Clockwork Percussionist': 29,
 'Collected Company': 30,
 'Come Back Wrong': 31,
 'Commune with Evil': 32,
 'Conductive Machete': 33,
 'Coordinated Clobbering': 34,
 'Cracked Skull': 35,
 'Creeping Peeper': 36,
 'Cryptid Inspector': 37,


I'll leave this code here, but I'll write a global function.

# Build `chosen` and `options` lists
In fact, I'll build a dict indexed by `draft_id` that contains the lists corresponding to each individual draft.

In [44]:
def count_to_list(row, prefix):
    """
    Turns a vector of counts into a list of card names, each one
    repeated as many times as the vector's entry. We obtain the
    name of the cards by extracting the names of the columns with
    non-zero value and removing the given column prefix from it.
    """
    # Filter only columns with the input prefix and tranpose
    df_prefix = row.filter(regex=prefix)
    df_prefix = df_prefix.transpose()

    # Get rows whose entry is not 0
    idx_orig = row.index[0]
    column_list = df_prefix[df_prefix[idx_orig] > 0].index

    # Remove prefix and add repetitions
    card_list = []
    for col_name in column_list:
        card_name = col_name.replace(prefix, "")
        repetitions = row.loc[idx_orig, col_name]

        card_list.extend([card_name] * repetitions)

    return card_list

In [68]:
# Draft data
num_packs = 3
num_picks = 14

# Get unique ids
draft_ids = draftdata["draft_id"].unique()

# Get columns with the player's options
prefix_pack = "pack_card_"
pack_columns = draftdata.filter(regex=prefix_pack).columns
pack_columns = list(pack_columns)

# Get columns with the player's pool of cards
prefix_pool = "pool_"
pool_columns = draftdata.filter(regex=prefix_pool).columns
pool_columns = list(pool_columns)

# Get only card names
card_names = [name.replace(prefix_pack, "") for name in pack_columns]

# Compile data for each draft_id
drafts = {}
for i, id in enumerate(draft_ids):
    time_start = time()

    # Get draft info for id
    data_id = draftdata.loc[draftdata["draft_id"] == id, :]

    # Check that we have the right amount of data
    num_rows = data_id.shape[0]
    if num_rows != num_packs * num_picks:
        print(f"{i+1}/{len(draft_ids)}", end=": ")
        print(
            f"Draft incomplete. Only {num_rows} out of {num_packs*num_picks} rows. Skipping id {id}."
        )
        continue

    # Build iterators to extract information in turn order
    draft_turns = product(range(num_packs), range(num_picks))

    chosen = []
    options = []
    pool = []
    for pack_idx, pick_idx in draft_turns:
        # Get row for the turn by filtering pack number, pick number, and draft id
        df_turn = draftdata[
            (draftdata["draft_id"] == id)
            & (draftdata["pack_number"] == pack_idx)
            & (draftdata["pick_number"] == pick_idx)
        ]

        # Get pick, cards in pack, and cards in pool
        df_index = df_turn.index[0]
        pick = df_turn.at[df_index, "pick"]
        cards_in_pack = count_to_list(df_turn, prefix_pack)
        cards_in_pool = count_to_list(df_turn, prefix_pool)

        # Store results as indices
        chosen.append(card_to_idx[pick])
        options.append([card_to_idx[card] for card in cards_in_pack])
        pool.append([card_to_idx[card] for card in cards_in_pool])

    # Store results for the id
    drafts[id] = (chosen, options, pool)

    time_end = time()
    dt = time_end - time_start
    print(f"{i+1}/{len(draft_ids)}: {np.round(dt,3)}")

1/239: 0.068
2/239: 0.071
3/239: 0.068
4/239: 0.068
5/239: 0.068
6/239: 0.067
7/239: 0.067
8/239: 0.067
9/239: 0.067
10/239: 0.066
11/239: 0.068
12/239: 0.068
13/239: 0.072
14/239: 0.069
15/239: 0.068
16/239: 0.068
17/239: 0.068
18/239: 0.069
19/239: 0.072
20/239: 0.068
21/239: 0.067
22/239: 0.069
23/239: 0.068
24/239: 0.073
25/239: 0.07
26/239: 0.066
27/239: 0.067
28/239: 0.068
29/239: 0.073
30/239: 0.067
31/239: 0.071
32/239: 0.07
33/239: 0.068
34/239: 0.067
35/239: 0.068
36/239: 0.07
37/239: 0.069
38/239: 0.069
39/239: 0.069
40/239: 0.07
41/239: 0.066
42/239: 0.068
43/239: 0.07
44/239: 0.069
45/239: 0.067
46/239: 0.068
47/239: 0.08
48/239: 0.067
49/239: 0.067
50/239: 0.067
51/239: 0.07
52/239: 0.067
53/239: 0.068
54/239: 0.071
55/239: 0.067
56/239: 0.068
57/239: 0.067
58/239: 0.078
59/239: 0.066
60/239: 0.071
61/239: 0.068
62/239: 0.069
63/239: 0.068
64/239: 0.067
65/239: 0.069
66/239: 0.068
67/239: 0.072
68/239: 0.067
69/239: 0.072
70/239: 0.066
71/239: 0.068
72/239: 0.067
73/239: 

## Sanity check
Verify that list of chosen cards from turn `0` to turn `i` is the same as the pool in turn `i+1`.

In [83]:
all_equal = True
for id in draft_ids:
    if id in drafts.keys():
        chosen, options, pool = drafts[id]
    else:
        continue

    # Create a new copy of the pool with the chosen cards
    pool_2 = [[]]
    for card in chosen:
        # The pool in the next turn equals the current pool
        # plus the card added this turn
        pool_curr = pool_2[-1]
        pool_next = pool_curr + [card]

        # Sort to avoid order mismatches
        pool_2.append(sorted(pool_next))

    # The last entry is the final deck, not a pool, so we remove it
    _ = pool_2.pop(-1)

    # Compare pools
    pool_sort = [sorted(P) for P in pool]

    # Display any error
    if pool_2 != pool_sort:
        print(" ---------------------- ")
        print("The pools are different")
        print(" ---------------------- ")
        print("id:", id)
        print()

        print("Original pool:")
        print(pool_sort)
        print()

        print("Reconstructed pool:")
        print(pool_2)
        print()

        all_equal = False

if all_equal:
    print("No errors!")

No errors!


## Examples of output

In [84]:
id = draft_ids[0]
chosen, options, pool = drafts[id]

print("Number of chosen cards:", len(chosen))
print("Chosen cards:")
print(chosen)

Number of chosen cards: 42
Chosen cards:
[123, 89, 73, 73, 254, 44, 106, 101, 122, 202, 158, 273, 14, 270, 186, 192, 124, 5, 271, 160, 29, 209, 18, 122, 18, 106, 22, 194, 15, 74, 49, 18, 15, 150, 195, 179, 202, 82, 192, 104, 29, 64]


In [85]:
# Displaying the player's first 5 picks
for idc in range(5):
    print(f"Turn {idc}")
    print("Chosen: ", chosen[idc])
    print("Options:", options[idc])
    print("Pool:   ", pool[idc])
    print()

Turn 0
Chosen:  123
Options: [39, 58, 78, 90, 122, 123, 154, 160, 180, 190, 222, 246, 256, 284]
Pool:    []

Turn 1
Chosen:  89
Options: [89, 90, 115, 146, 149, 165, 202, 208, 210, 217, 225, 270, 282]
Pool:    [123]

Turn 2
Chosen:  73
Options: [8, 11, 17, 73, 93, 112, 158, 172, 190, 230, 253, 264]
Pool:    [89, 123]

Turn 3
Chosen:  73
Options: [8, 33, 38, 64, 73, 86, 91, 112, 234, 253, 273]
Pool:    [73, 89, 123]

Turn 4
Chosen:  254
Options: [14, 33, 42, 115, 119, 129, 147, 217, 253, 254]
Pool:    [73, 73, 89, 123]



In [86]:
# Display names of chosen cards using idx_to_card
for idc in chosen:
    print(idx_to_card[idc])

Infernal Phantom
Floodpits Drowner
Fear of Being Hunted
Fear of Being Hunted
Undead Sprinter
Dashing Bloodsucker
Grab the Prize
Glassworks // Shattered Yard
Impossible Inferno
Ripchain Razorkin
Most Valuable Slayer
Vicious Clown
Bedhead Beastie
Vanish from Sight
Piggy Bank
Ragged Playmate
Innocuous Rat
Appendage Amalgam
Vengeful Possession
Murder
Clockwork Percussionist
Sawblade Skinripper
Boilerbilges Ripper
Impossible Inferno
Boilerbilges Ripper
Grab the Prize
Cackling Slasher
Raucous Carnival
Betrayer's Bargain
Fear of Burning Alive
Derelict Attic // Widow's Walk
Boilerbilges Ripper
Betrayer's Bargain
Marvin, Murderous Mimic
Razorkin Hordecaller
Painter's Studio // Defaced Gallery
Ripchain Razorkin
Fear of Lost Teeth
Ragged Playmate
Glimmerlight
Clockwork Percussionist
Enter the Enigma


In [87]:
# Getting a card's index using card_to_idx
card_to_idx["Enter the Enigma"]

64